# Customize state
- Doc: https://langchain-ai.github.io/langgraph/tutorials/get-started/5-customize-state/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")

## 1. Add keys to the state

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


# 에이전트의 상태 정의 클래스
class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str      # 이름 (새로 추가됨)
    birthday: str  # 생일 (새로 추가됨)

## 2. Update the state inside the tool

In [ ]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

from langgraph.types import Command, interrupt

# human_assistance 도구 정의 (인간의 도움을 요청하는 도구)
@tool
def human_assistance(
    name: str,
    birthday: str,
    # tool_call_id: 도구 호출의 고유한 DI, 도구 호출과 그 결과를 추적하기 위함 (자동 생성)
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    # 인간에게 질문과 현재 상태(name, birthday)를 전달 후 응답을 기다림
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # 응답이 y로 시작하면 현재 상태 유지
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # 그렇지 않으면 인간이 제공한 수정된 name, birthday 사용
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # 상태 업데이트: ToolMessage와 함께 name, birthday 갱신
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # Command 객체를 반환하여 상태 업데이트 명령 전달
    return Command(update=state_update)

In [ ]:
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

# 웹 검색을 도구
tool = TavilySearch(max_results=5)
# 도구 리스트
tools = [tool, human_assistance]

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # 병렬 도구 호출 비활성화: 인터럽트 후 툴 중복 호출 방지
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

# tool 노드 워크플로우에 추가
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# 조건부 라우팅: tools로 이동하거나 END로 이동 (종료)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")

# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")

In [ ]:
memory = MemorySaver()
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성.
# memory checkpointer 사용
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

try:
    # 워크플로우 그래프를 이미지로 시각화하여 출력
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## 3. Prompt the chatbot

In [ ]:
user_input = (
    "이순신 장군이 언제 태아나셨는지 알아봐줘. "
    "답변하기 전 human_assistance 도구를 사용해서 검토해줘."
)
# 대화 세션을 구분하기 위한 설정
config = {"configurable": {"thread_id": "1"}}

# 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",  # values: 각 노드가 실행을 마친 후 결과를 스트리밍
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

## 4. Add human assistance

In [ ]:
# Command 객체의 resume에 인간 응답 저장
human_command = Command(
    resume={
        "name": "손흥민",
        "birthday": "1992년 7월 8일"
    },
)

# 워크플로우를 스트리밍 모드로 실행 (인간 응답 처리)
events = graph.stream(
    human_command,
    config,
    stream_mode="values",  # values: 각 노드가 실행을 마친 후 결과를 스트리밍
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
# config에 해당하는 워크플로우의 상태 스냅샷 조회
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

## 5. Manually update the state

In [ ]:
# LangGraph 워크플로우의 상태를 업데이트
graph.update_state(config, {"name": "류현진"})

## 6. View the new value

In [ ]:
# config에 해당하는 워크플로우의 상태 스냅샷 조회
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}